<a href="https://colab.research.google.com/github/JohnMoriarty009/sales_tracking.py/blob/main/Sales%20database%20code%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Très bien ! Passons à la troisième étape : afficher toutes les ventes enregistrées dans la base de données.

Nous allons ajouter une nouvelle fonction, `view_sales`, à notre script. Cette fonction se connectera à la base de données, récupérera toutes les lignes de la table `sales` et les affichera de manière lisible.

Voici le code mis à jour, incluant la fonction `view_sales` :

In [1]:
import sqlite3
import os
from datetime import datetime

# Define the database file name
DATABASE_NAME = 'sales_tracking.db'

def initialize_database():
    """Connects to the SQLite database and creates the sales table if it doesn't exist."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sales (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_name TEXT NOT NULL,
                customer_contact TEXT,
                sale_date TEXT NOT NULL,
                product_name TEXT NOT NULL,
                price REAL NOT NULL,
                quantity INTEGER NOT NULL,
                total_revenue REAL NOT NULL
            )
        ''')

        conn.commit()
        # print(f"Database '{DATABASE_NAME}' initialized and 'sales' table ready.") # Keep output clean for main execution

    except sqlite3.Error as e:
        print(f"Database error during initialization: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

def add_sale(customer_name, customer_contact, sale_date, product_name, price, quantity):
    """Adds a new sale record to the database."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        total_revenue = price * quantity

        cursor.execute('''
            INSERT INTO sales (customer_name, customer_contact, sale_date, product_name, price, quantity, total_revenue)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (customer_name, customer_contact, sale_date, product_name, price, quantity, total_revenue))

        conn.commit()
        print(f"Sale added for {customer_name} ({product_name}, {quantity} pcs).") # More specific success message

    except sqlite3.Error as e:
        print(f"Database error when adding sale: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

def view_sales():
    """Retrieves and displays all sale records from the database."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Select all columns for all rows in the sales table
        cursor.execute("SELECT id, customer_name, customer_contact, sale_date, product_name, price, quantity, total_revenue FROM sales")

        # Fetch all results
        sales_records = cursor.fetchall()

        if not sales_records:
            print("\nNo sales records found yet.")
            return

        print("\n--- All Sales Records ---")
        # Print a header
        print(f"{'ID':<4} | {'Customer':<20} | {'Contact':<20} | {'Date':<10} | {'Product':<25} | {'Price':<8} | {'Qty':<4} | {'Total':<10}")
        print("-" * 115) # Separator line

        # Iterate over the fetched rows and print each sale
        for record in sales_records:
            # Unpack the tuple into individual variables
            id, customer_name, customer_contact, sale_date, product_name, price, quantity, total_revenue = record
            print(f"{id:<4} | {customer_name:<20} | {customer_contact:<20} | {sale_date:<10} | {product_name:<25} | {price:<8.2f} | {quantity:<4} | {total_revenue:<10.2f}")
        print("-" * 115) # Separator line

    except sqlite3.Error as e:
        print(f"Database error when viewing sales: {e}")

    finally:
        if conn:
            conn.close()

# --- How to run this step ---
if __name__ == "__main__":
    # First, ensure the database and table are initialized
    initialize_database()
    print(f"Ensured database '{DATABASE_NAME}' is initialized.") # Add a message here

    # --- Add some sample sales (you can comment these out after the first run) ---
    print("\nAdding sample sales...")
    add_sale("Alice Dubois", "alice.d@email.com", "2023-10-27", "Ordinateur Portable", 850.00, 1)
    add_sale("Bob Martin", "06 01 02 03 04", "2023-10-27", "Clavier Mécanique", 75.50, 2)
    add_sale("Alice Dubois", "alice.d@email.com", "2023-10-28", "Souris sans fil", 25.00, 3)
    add_sale("Charlie Dupont", None, "2023-10-28", "Écran 27 pouces", 250.00, 1) # Example with no contact

    # --- Now, view all sales ---
    print("\n--- Viewing all sales records ---")
    view_sales()

Ensured database 'sales_tracking.db' is initialized.

Adding sample sales...
Sale added for Alice Dubois (Ordinateur Portable, 1 pcs).
Sale added for Bob Martin (Clavier Mécanique, 2 pcs).
Sale added for Alice Dubois (Souris sans fil, 3 pcs).
Sale added for Charlie Dupont (Écran 27 pouces, 1 pcs).

--- Viewing all sales records ---

--- All Sales Records ---
ID   | Customer             | Contact              | Date       | Product                   | Price    | Qty  | Total     
-------------------------------------------------------------------------------------------------------------------
1    | Alice Dubois         | alice.d@email.com    | 2023-10-27 | Ordinateur Portable       | 850.00   | 1    | 850.00    
2    | Bob Martin           | 06 01 02 03 04       | 2023-10-27 | Clavier Mécanique         | 75.50    | 2    | 151.00    
3    | Alice Dubois         | alice.d@email.com    | 2023-10-28 | Souris sans fil           | 25.00    | 3    | 75.00     


TypeError: unsupported format string passed to NoneType.__format__

**Explication de la nouvelle fonction `view_sales` :**

1.  `def view_sales():`: On définit la fonction `view_sales`. Elle n'a pas besoin d'arguments car elle affiche *toutes* les ventes.
2.  `conn = sqlite3.connect(DATABASE_NAME)`: On se connecte à la base de données.
3.  `cursor = conn.cursor()`: On obtient un curseur.
4.  `cursor.execute("SELECT id, customer_name, ..., total_revenue FROM sales")`: C'est la commande SQL `SELECT` pour récupérer des données. `SELECT * FROM sales` sélectionnerait toutes les colonnes, mais il est généralement préférable de lister explicitement les colonnes que l'on veut (`id, customer_name, ...`). `FROM sales` spécifie la table d'où récupérer les données.
5.  `sales_records = cursor.fetchall()`: Après avoir exécuté une requête `SELECT`, vous devez récupérer les résultats. `cursor.fetchall()` récupère *toutes* les lignes du résultat de la requête et les renvoie sous forme de liste. Chaque élément de la liste est un tuple représentant une ligne de la table.
6.  `if not sales_records:`: On vérifie si la liste `sales_records` est vide. Si c'est le cas, cela signifie qu'il n'y a pas encore de ventes dans la base de données, et on affiche un message informatif.
7.  `print(...)`: Si des enregistrements sont trouvés, on affiche un en-tête pour les colonnes pour rendre la sortie plus lisible. J'utilise ici la mise en forme de chaînes (`f"..."`) avec des spécificateurs d'alignement et de largeur (par exemple, `{<20}` pour aligner à gauche sur 20 caractères) pour créer une sortie tabulaire.
8.  `for record in sales_records:`: On boucle à travers chaque tuple (chaque enregistrement de vente) dans la liste `sales_records`.
9.  `id, customer_name, ..., total_revenue = record`: On "déballe" le tuple `record` dans des variables distinctes pour faciliter l'accès aux différentes valeurs.
10. `print(...)`: On affiche les informations de chaque vente en utilisant la même mise en forme pour les colonnes que dans l'en-tête. Notez l'utilisation de `.2f` pour formater les nombres décimaux (`price`, `total_revenue`) avec deux chiffres après la virgule.
11. Le bloc `try...except...finally`: Gère les erreurs et ferme la connexion.

**Comment tester ce code :**

1.  Remplacez le contenu de votre fichier `sales_db.py` par ce nouveau code.
2.  Exécutez le script : `python sales_db.py`
3.  Le script va d'abord ajouter les exemples de ventes (si ce n'est pas déjà fait), puis il appellera `view_sales()`.
4.  Vous devriez voir la liste des ventes affichée dans votre terminal sous forme de tableau.

Nous pouvons maintenant initialiser la base de données, ajouter des ventes et les visualiser. La prochaine étape logique est de pouvoir calculer le chiffre d'affaires total.

Êtes-vous prêt pour cette quatrième étape ?